In [2]:
import pandas as pd
import re
import datetime
import numpy as np

# 2019 Data Preprocess

In [3]:
pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")

/var/folders/08/6rx4zjtn10557fvbrhbl5ykw0000gn/T/ipykernel_65763/20673864.py:1: DtypeWarning: Columns (167,168,169,170,175,176,179,180,183,184,189,190,191,192,195,196,204,205,206,219,220,221,222,223,224,227,228,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")


In [4]:
highlights19 = pd.read_csv("./2019_NFL_highlight_transcript - Sheet1.csv")
highlights19 = highlights19[~highlights19["inning"].isna()]

In [5]:
relevant_cols = pbp_data_19.drop(columns=(list(pbp_data_19.iloc[:,58:110].columns) + ["desc","home_timeouts_remaining","home_timeouts_remaining","timeout","timeout_team","play_id","game_id","game_date"])).columns

In [6]:
pbp_data_19.columns

Index(['play_id', 'game_id', 'home_team', 'away_team', 'posteam',
       'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date',
       ...
       'penalty_player_id', 'penalty_player_name', 'penalty_yards',
       'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type',
       'defensive_two_point_attempt', 'defensive_two_point_conv',
       'defensive_extra_point_attempt', 'defensive_extra_point_conv'],
      dtype='object', length=256)

In [7]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [8]:
highlights19["intervals"] = highlights19["inning"].apply(process_inning_time)
highlights19["date"] = pd.to_datetime(highlights19["date"])
pbp_data_19["game_date"] = pd.to_datetime(pbp_data_19["game_date"])

In [9]:
merged19 = pbp_data_19.merge(highlights19,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [10]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [11]:
plays19 = merged19[merged19.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [12]:
touches = plays19[plays19["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [13]:
mi_index = touches[touches == 1].index

In [14]:
plays19 = plays19[(plays19["touchdown"] == 1) & (plays19["intervals"].isin(mi_index))]

In [15]:
X = plays19[relevant_cols.to_list()+["transcript"]]

In [16]:
# transcript_grouped = X.groupby("transcript").groups
# transcript_groups = list(transcript_grouped.keys())

In [21]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[relevant_cols].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["transcript"].iloc[i]}]

In [22]:
# ret_list = []
# for i in range(len(transcript_groups)):
#     transcript_group = X.loc[transcript_grouped[transcript_groups[i]]]
#     ret_list += [{"json":transcript_group.loc[transcript_group["touchdown"]==1,relevant_cols].replace(0,np.nan).dropna(axis=1).iloc[0].to_json(),
#                   "transcript":transcript_group["transcript"].iloc[0]}]

In [23]:
ret_list[4]["transcript"]

'first and 10 25 yard line off the fake to goldman! Jones pass is caught by Evan Engram still going! Down the sideline! Evan Edward will take this all the way! What a great read by Daniel Jones! More importantly, what a great read by Evan Engram!'

In [43]:
my_sample = pd.Series(ret_list).sample(4)
my_sample

25    {'json': '{"home_team":"SF","away_team":"SEA",...
21    {'json': '{"home_team":"TEN","away_team":"LAC"...
3     {'json': '{"home_team":"TB","away_team":"NYG",...
2     {'json': '{"home_team":"TB","away_team":"NYG",...
dtype: object

In [32]:
plays19[plays19["fumble_recovery_1_player_name"] == "G.Ifedi"]["transcript"].iloc[0]

"play action off of this on second and seven and they get to him and they do so by bringing Williams and then the ball comes out! And picking it up is bucker! And we've got a San Francisco touchdown! Deforest Buckner! great football awareness!"

In [47]:
my_sample.iloc[3]

{'json': '{"home_team":"TB","away_team":"NYG","posteam":"NYG","posteam_type":"away","defteam":"TB","side_of_field":"TB","yardline_100":7.0,"quarter_seconds_remaining":606.0,"half_seconds_remaining":606.0,"game_seconds_remaining":2406.0,"game_half":"Half1","drive":4,"sp":1,"qtr":2,"down":2.0,"goal_to_go":1.0,"time":"10:06","yrdln":"TB 7","ydstogo":7,"ydsnet":75,"play_type":"run","yards_gained":7,"shotgun":1,"run_location":"right","run_gap":"end","away_timeouts_remaining":3,"td_team":"NYG","posteam_timeouts_remaining":3.0,"defteam_timeouts_remaining":3.0,"total_home_score":12,"total_away_score":9,"posteam_score":3.0,"defteam_score":12.0,"score_differential":-9.0,"posteam_score_post":9.0,"defteam_score_post":12.0,"score_differential_post":-3.0,"first_down_rush":1.0,"rush_attempt":1.0,"touchdown":1.0,"rush_touchdown":1.0,"rusher_player_id":"00-0035710","rusher_player_name":"D.Jones"}',
 'transcript': 'the second and goal this is Jones. in side the 5, to the end zone! Touchdown! He knew tha

In [129]:
pd.Series(ret_list).sample(1,random_state=57).iloc[0]

{'json': '{"home_team":"BAL","away_team":"CLE","posteam":"CLE","posteam_type":"away","defteam":"BAL","side_of_field":"BAL","yardline_100":2.0,"quarter_seconds_remaining":58.0,"half_seconds_remaining":958.0,"game_seconds_remaining":958.0,"game_half":"Half2","drive":14,"sp":1,"qtr":3,"down":1.0,"goal_to_go":1.0,"time":"00:58","yrdln":"BAL 02","ydstogo":2,"ydsnet":74,"play_type":"run","yards_gained":2,"run_location":"left","run_gap":"tackle","away_timeouts_remaining":3,"td_team":"CLE","posteam_timeouts_remaining":3.0,"defteam_timeouts_remaining":3.0,"total_home_score":10,"total_away_score":23,"posteam_score":17.0,"defteam_score":10.0,"score_differential":7.0,"posteam_score_post":23.0,"defteam_score_post":10.0,"score_differential_post":13.0,"first_down_rush":1.0,"rush_attempt":1.0,"touchdown":1.0,"rush_touchdown":1.0,"rusher_player_id":"00-0034791","rusher_player_name":"N.Chubb"}',
 'transcript': 'Nick Chubb trying ot find the endzone and he does! Touchdown Cleveland.'}